In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
import json
import math

In [2]:
complete_stations = pd.read_csv("complete_air_quality_stations.csv", index_col=0)
complete_stations = complete_stations.replace({np.nan: None})
complete_stations

,IdSensore,NomeTipoSensore,UnitaMisura,Idstazione,NomeStazione,Quota,Provincia,Comune,Storico,DataStart,DataStop,Utm_Nord,UTM_Est,lat,lng
0,10431,Ossidi di Azoto,µg/m³,1264,Sondrio v.Paribelli,290.0,SO,Sondrio,N,2008-11-11,None,5113073,567873,46.167852,9.87921
1,9968,Ossidi di Azoto,µg/m³,679,Valmadrera v.Pozzi,247.0,LC,Valmadrera,N,2006-02-16,None,5076576,527307,45.842207,9.351658
2,5543,Biossido di Azoto,µg/m³,540,Garbagnate Milanese v. Villoresi,175.0,MI,Garbagnate Milanese,S,1992-12-11,2018-07-30,5046467,506105,45.571716,9.078242
3,20197,PM10 (SM2005),µg/m³,598,San Rocco al Porto v.Matteotti,50.0,LO,San Rocco al Porto,N,2002-01-01,None,4992297,555154,45.081987,9.700788
4,5611,Biossido di Azoto,µg/m³,608,Codogno v.Trento,59.0,LO,Codogno,N,2000-01-13,None,5000892,554872,45.159373,9.698148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,20532,PM10 (SM2005),µg/m³,569,Sondrio v.Mazzini,307.0,SO,Sondrio,N,None,None,5113078,567173,46.167971,9.870146
978,20569,Monossido di Carbonio,mg/m³,1801,Brescia v.Tartaglia,151.0,BS,Brescia,N,2023-01-03,None,5043948,594556,45.542648,10.211205
979,20570,Ozono,µg/m³,659,Odolo v.Praes,349.0,BS,Odolo,N,2023-01-03,None,5055661,608260,45.646052,10.389297
980,20572,PM10 (SM2005),µg/m³,564,Erba v. Battisti,279.0,CO,Erba,N,None,None,5072803,517232,45.808574,9.221779


In [5]:
base_url = "http://127.0.0.1:8000/api/collections/airqualitysensor"
body = {
    "bulk": True,
    "items": []
}

station_to_add = complete_stations.copy()
for i, sensor in station_to_add.iterrows():
    item = {
        "sensor_id": sensor.IdSensore,
        "sensor_type": sensor.NomeTipoSensore,
        "measurement_unit": sensor.UnitaMisura,
        "station_id": sensor.Idstazione,
        "station_name": sensor.NomeStazione,
        "altitude": sensor.Quota,
        "province": sensor.Provincia,
        "comune": sensor.Comune,
        "is_historical": sensor.Storico == 'S',
        "date_start": sensor.DataStart, #YYYY-MM-DD
        "date_stop": sensor.DataStop, #YYYY-MM-DD
        "utm_north": sensor.Utm_Nord,
        "utm_east": sensor.UTM_Est,
        "latitude": sensor.lat,
        "longitude": sensor.lng,
        "location": f"POINT ({sensor.lng} {sensor.lat})"
    }
    body["items"].append(item)
    
body_json = json.dumps(body)

#uncomment for creating the stations
req = requests.post(base_url, body_json)
if req.status_code == 500:
    print(body_json)
print(f'request {i} - {req.status_code}')


request 981 - 200
